### Use catalog and schema

In [0]:
%sql
USE CATALOG adb_nhs_eng_lakehouse;
USE SCHEMA gold;

### Incremental Loading for gold_monthly_national_kpis

In [0]:
%sql
MERGE INTO gold.monthly_national_kpis tgt
USING (

    SELECT
        s.period,

        /* Base totals */
        SUM(
            s.ae_attendances_type_1 +
            s.ae_attendances_type_2 +
            s.ae_attendances_other
        ) AS total_ae_attendances,

        SUM(
            s.emergency_admissions_ae_type_1 +
            s.emergency_admissions_ae_type_2 +
            s.emergency_admissions_other_ae +
            s.other_emergency_admissions
        ) AS total_emergency_admissions,

        SUM(
            s.attendances_over_4hrs_type_1 +
            s.attendances_over_4hrs_type_2 +
            s.attendances_over_4hrs_other
        ) AS total_over_4hrs_attendances,

        SUM(s.patients_12hrs_plus_dta) AS total_12hrs_plus_patients,

        /* Derived KPIs */
        ROUND(
            SUM(
                s.attendances_over_4hrs_type_1 +
                s.attendances_over_4hrs_type_2 +
                s.attendances_over_4hrs_other
            ) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS pct_over_4hrs_attendances,

        ROUND(
            SUM(s.patients_12hrs_plus_dta) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS pct_12hrs_plus_patients,

        ROUND(
            SUM(
                s.emergency_admissions_ae_type_1 +
                s.emergency_admissions_ae_type_2 +
                s.emergency_admissions_other_ae +
                s.other_emergency_admissions
            ) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS admission_conversion_rate

    FROM adb_nhs_eng_lakehouse.silver.ae_attendances_silver s

    /* Only bring new periods */
    LEFT ANTI JOIN gold.monthly_national_kpis g
        ON s.period = g.period

    GROUP BY s.period

) src
ON tgt.period = src.period

WHEN NOT MATCHED THEN
  INSERT (
    period,
    total_ae_attendances,
    total_emergency_admissions,
    total_over_4hrs_attendances,
    total_12hrs_plus_patients,
    pct_over_4hrs_attendances,
    pct_12hrs_plus_patients,
    admission_conversion_rate
  )
  VALUES (
    src.period,
    src.total_ae_attendances,
    src.total_emergency_admissions,
    src.total_over_4hrs_attendances,
    src.total_12hrs_plus_patients,
    src.pct_over_4hrs_attendances,
    src.pct_12hrs_plus_patients,
    src.admission_conversion_rate
  );

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


### Incremental Loading for gold_monthly_org_kpis

In [0]:
%sql
MERGE INTO gold.monthly_org_kpis tgt
USING (

    SELECT
        s.period,
        s.org_code,
        s.parent_org,
        s.org_name,

        /* Base totals */
        SUM(
            s.ae_attendances_type_1 +
            s.ae_attendances_type_2 +
            s.ae_attendances_other
        ) AS total_ae_attendances,

        SUM(
            s.emergency_admissions_ae_type_1 +
            s.emergency_admissions_ae_type_2 +
            s.emergency_admissions_other_ae +
            s.other_emergency_admissions
        ) AS total_emergency_admissions,

        SUM(
            s.attendances_over_4hrs_type_1 +
            s.attendances_over_4hrs_type_2 +
            s.attendances_over_4hrs_other
        ) AS total_over_4hrs_attendances,

        SUM(s.patients_12hrs_plus_dta) AS total_12hrs_plus_patients,

        /* Derived KPIs */
        ROUND(
            SUM(
                s.attendances_over_4hrs_type_1 +
                s.attendances_over_4hrs_type_2 +
                s.attendances_over_4hrs_other
            ) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS pct_over_4hrs_attendances,

        ROUND(
            SUM(s.patients_12hrs_plus_dta) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS pct_12hrs_plus_patients,

        ROUND(
            SUM(
                s.emergency_admissions_ae_type_1 +
                s.emergency_admissions_ae_type_2 +
                s.emergency_admissions_other_ae +
                s.other_emergency_admissions
            ) /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ) * 100, 2
        ) AS admission_conversion_rate

    FROM adb_nhs_eng_lakehouse.silver.ae_attendances_silver s

    /* Only bring rows not already in Gold */
    LEFT ANTI JOIN gold.monthly_org_kpis g
        ON s.period = g.period
       AND s.org_code = g.org_code

    GROUP BY
        s.period,
        s.org_code,
        s.parent_org,
        s.org_name

) src
ON tgt.period = src.period
AND tgt.org_code = src.org_code

WHEN NOT MATCHED THEN
  INSERT (
    period,
    org_code,
    parent_org,
    org_name,
    total_ae_attendances,
    total_emergency_admissions,
    total_over_4hrs_attendances,
    total_12hrs_plus_patients,
    pct_over_4hrs_attendances,
    pct_12hrs_plus_patients,
    admission_conversion_rate
  )
  VALUES (
    src.period,
    src.org_code,
    src.parent_org,
    src.org_name,
    src.total_ae_attendances,
    src.total_emergency_admissions,
    src.total_over_4hrs_attendances,
    src.total_12hrs_plus_patients,
    src.pct_over_4hrs_attendances,
    src.pct_12hrs_plus_patients,
    src.admission_conversion_rate
  );

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


### Incremental Loading for gold_monthly_performance_indicators

In [0]:
%sql
MERGE INTO gold.monthly_performance_pressure_kpis tgt
USING (

    SELECT
        s.period,

        /* Base totals */
        SUM(
            s.ae_attendances_type_1 +
            s.ae_attendances_type_2 +
            s.ae_attendances_other
        ) AS total_attendances,

        SUM(
            s.attendances_over_4hrs_type_1 +
            s.attendances_over_4hrs_type_2 +
            s.attendances_over_4hrs_other
        ) AS total_over_4hrs,

        SUM(s.patients_12hrs_plus_dta) AS total_12hrs_plus,

        SUM(
            s.emergency_admissions_ae_type_1 +
            s.emergency_admissions_ae_type_2 +
            s.emergency_admissions_other_ae +
            s.other_emergency_admissions
        ) AS total_emergency_admissions,

        SUM(s.ae_attendances_type_1) AS type1_attendances,

        /* Derived KPIs */
        ROUND(
            (
                SUM(
                    s.attendances_over_4hrs_type_1 +
                    s.attendances_over_4hrs_type_2 +
                    s.attendances_over_4hrs_other
                )
                + SUM(s.patients_12hrs_plus_dta)
            )
            /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ), 4
        ) AS pressure_index,

        ROUND(
            SUM(
                s.emergency_admissions_ae_type_1 +
                s.emergency_admissions_ae_type_2 +
                s.emergency_admissions_other_ae +
                s.other_emergency_admissions
            )
            /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ), 4
        ) AS admission_conversion_rate,

        ROUND(
            SUM(s.ae_attendances_type_1)
            /
            NULLIF(
                SUM(
                    s.ae_attendances_type_1 +
                    s.ae_attendances_type_2 +
                    s.ae_attendances_other
                ), 0
            ), 4
        ) AS type1_dependency

    FROM adb_nhs_eng_lakehouse.silver.ae_attendances_silver s

    /* Only new periods */
    LEFT ANTI JOIN gold.monthly_performance_pressure_kpis g
        ON s.period = g.period

    GROUP BY s.period

) src
ON tgt.period = src.period

WHEN NOT MATCHED THEN
INSERT (
    period,
    total_attendances,
    total_over_4hrs,
    total_12hrs_plus,
    total_emergency_admissions,
    type1_attendances,
    pressure_index,
    admission_conversion_rate,
    type1_dependency
)
VALUES (
    src.period,
    src.total_attendances,
    src.total_over_4hrs,
    src.total_12hrs_plus,
    src.total_emergency_admissions,
    src.type1_attendances,
    src.pressure_index,
    src.admission_conversion_rate,
    src.type1_dependency
);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


### Incremental Loading for gold_monthly_trend_kpis

In [0]:
%sql
MERGE INTO gold.monthly_trend_kpis tgt
USING (

    WITH base AS (
        SELECT
            s.period,
            SUM(
                s.ae_attendances_type_1 +
                s.ae_attendances_type_2 +
                s.ae_attendances_other
            ) AS total_attendances,

            SUM(
                s.attendances_over_4hrs_type_1 +
                s.attendances_over_4hrs_type_2 +
                s.attendances_over_4hrs_other
            ) AS total_over_4hrs
        FROM adb_nhs_eng_lakehouse.silver.ae_attendances_silver s
        GROUP BY s.period
    ),

    ranked AS (
        SELECT *,
               DENSE_RANK() OVER (ORDER BY period DESC) AS rn
        FROM base
    )

    SELECT
        period,
        total_attendances,
        total_over_4hrs,

        total_attendances
          - LAG(total_attendances) OVER (ORDER BY period)
          AS month_on_month_attendance_change,

        total_over_4hrs
          - LAG(total_over_4hrs) OVER (ORDER BY period)
          AS month_on_month_wait_change,

        ROUND(
            AVG(total_attendances)
            OVER (ORDER BY period ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),
            0
        ) AS rolling_3_month_avg_attendances

    FROM ranked
    WHERE rn <= 3   -- only latest 3 months

) src
ON tgt.period = src.period

WHEN MATCHED THEN
UPDATE SET
    total_attendances = src.total_attendances,
    total_over_4hrs = src.total_over_4hrs,
    month_on_month_attendance_change = src.month_on_month_attendance_change,
    month_on_month_wait_change = src.month_on_month_wait_change,
    rolling_3_month_avg_attendances = src.rolling_3_month_avg_attendances

WHEN NOT MATCHED THEN
INSERT (
    period,
    total_attendances,
    total_over_4hrs,
    month_on_month_attendance_change,
    month_on_month_wait_change,
    rolling_3_month_avg_attendances
)
VALUES (
    src.period,
    src.total_attendances,
    src.total_over_4hrs,
    src.month_on_month_attendance_change,
    src.month_on_month_wait_change,
    src.rolling_3_month_avg_attendances
);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,3,0,0
